In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df1 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-01.txt.gz", error_bad_lines=False, sep="\t")
df2 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-02.txt.gz", error_bad_lines=False, sep="\t")
df3 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-03.txt.gz", error_bad_lines=False, sep="\t")
df4 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-04.txt.gz", error_bad_lines=False, sep="\t")
df5 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-05.txt.gz", error_bad_lines=False, sep="\t")
df6 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-06.txt.gz", error_bad_lines=False, sep="\t")
df7 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-07.txt.gz", error_bad_lines=False, sep="\t")
df8 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-08.txt.gz", error_bad_lines=False, sep="\t")
df9 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-09.txt.gz", error_bad_lines=False, sep="\t")
df10 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-10.txt.gz", error_bad_lines=False, sep="\t")
df = df1.append([df2,df3,df4,df5,df6,df7,df8,df9,df10], ignore_index=True)

/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df.drop_duplicates(['AnonID', 'Query'], keep='first')

In [4]:
# remove website query
df = df[~df['Query'].str.lower().str.contains('|'.join(["www.", ".com", ".uk", ".us", ".edu"]), na=False)]

In [5]:
df["QueryTime"] = pd.to_datetime(df["QueryTime"])

In [6]:
df = df.dropna(subset=['Query'])

In [7]:
df.columns

Index(['AnonID', 'Query', 'QueryTime', 'ItemRank', 'ClickURL'], dtype='object')

In [15]:
df.head()

,AnonID,Query,QueryTime,ItemRank,ClickURL
6,142,westchester.gov,2006-03-20 03:55:57,1,http://www.westchestergov.com
8,142,dfdf,2006-03-24 22:23:07,NaN,NaN
13,142,207 ad2d 530,2006-04-08 01:31:04,NaN,NaN
15,142,broadway.vera.org,2006-04-08 08:38:23,NaN,NaN
17,142,vera.org,2006-04-08 08:38:42,1,http://www.vera.org


In [27]:
ehow = df[df['ClickURL'].str.lower().str.contains("ehow", na=False)]

In [30]:
ehow = ehow[~ehow['Query'].str.lower().str.contains("how to", na=False)]

In [8]:
# '|'.join(["send", "group", "email"]
howto = df[df['Query'].str.lower().str.contains("how to", na=False)]

In [15]:
wiki = pd.read_csv("data/wikihowSep.csv", sep=",", error_bad_lines=False)

In [40]:
how = set([])
for i in wiki.title.str.lower().unique().tolist():
    how.add(regex.sub(' ', str(i)).replace("how to","")[1:])

In [8]:
from datetime import datetime  

In [9]:
howto["QueryTime"] = pd.to_datetime(howto["QueryTime"])

/Users/jarana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
import os
ids = [int(i.split("_")[0]) for i in os.listdir("data/all_ehow/")]

In [ ]:
%%time
from queue import Queue, Empty

window_size = 30
# jobs = Queue()
jobs = []
count = 0

for index, row in howto[~howto.AnonID.isin(ids)].iterrows():
    start_time = row['QueryTime'] - pd.Timedelta(minutes=window_size)
    stop_time = row['QueryTime'] + pd.Timedelta(minutes=window_size)
#     jobs.append([row['AnonID'], row['QueryTime'].strftime('%Y-%m-%d_%H:%M:%S'), start_time, stop_time])
#     jobs.put(index)
#     jobs.append(index)
    tmp = df[df.AnonID == row['AnonID']]
    ses = '\t'.join(tmp[tmp['QueryTime'].between(start_time, stop_time)]['Query'].tolist())
#     print(ses)
#     print('data/tmp/%d_%s.csv' % (row['AnonID'], row['QueryTime'].strftime('%Y-%m-%d_%H:%M:%S')))
    with open('data/all_ehow/%d_%s.csv' % (int(row['AnonID']), row['QueryTime'].strftime('%Y-%m-%d_%H:%M:%S')), 'w') as the_file:
        the_file.write('%s\n' % ses)
#     break
#     print(ses)
#     with open('data/session.csv', 'a+') as the_file:
#         the_file.write('%s\n' % ses)
